In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import random
import json
import gzip
import csv
from textblob import TextBlob


In [3]:
def parse_gzip_json_chunks(path, chunksize=10000):
    chunks = []
    with gzip.open(path, 'rt', encoding='utf-8') as g:
        for chunk in pd.read_json(g, lines=True, chunksize=chunksize):
            
            chunks.append(chunk)
    return pd.concat(chunks)
cali_df = parse_gzip_json_chunks('California.json.gz')

In [6]:
cali_df.to_csv('output.csv', escapechar='\\', quoting=csv.QUOTE_ALL,  index=False) 

In [7]:
cali_df

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
...,...,...,...,...,...,...,...,...
44476885,1.161470e+20,Stephanie Martinez,1509140119094,5,None,None,NaN,0x80c2c36f7efdaf83:0xd79b6337f9d1ea50
44476886,1.158846e+20,oswaldo cabrera,1568567374258,5,None,None,NaN,0x80c2c36f7efdaf83:0xd79b6337f9d1ea50
44476887,1.074242e+20,erick weber,1539293595641,5,None,None,NaN,0x80c2c36f7efdaf83:0xd79b6337f9d1ea50
44476888,1.042045e+20,M3XICANA1987,1560510170220,5,None,None,NaN,0x80c2c36f7efdaf83:0xd79b6337f9d1ea50
